# Carregar biblioteacas.

In [1]:
# Bibliotecas.
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
from sklearn.metrics import confusion_matrix
import numpy as np
from google.colab import files



# Selecionando e configurando um modelo pré-treinado.

In [2]:
# Selecionando um modelo pré-treinado, sem a camada do topo.
modelo_pre_treinado = tf.keras.applications.MobileNet(weights="imagenet", include_top=False,
                                                      input_shape=(224, 224, 3))

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Congelar o modelo pré-treinado.
modelo_pre_treinado.trainable = False

## Adicionando camadas.

In [4]:
# Ajustar número de dimenções.
x = modelo_pre_treinado.output
x = GlobalAveragePooling2D()(x)

# Adicionar camada intermediária e final.
x = Dense(50, activation="relu")(x)
previsoes = Dense(1, activation="sigmoid")(x)

# Criar modelo final.
modelo = Model(inputs= modelo_pre_treinado.input, outputs=previsoes)


# Baixando dataset (cats_vs_dogs)

In [5]:
# Ver informações sobre o dataset cats-vs-dogs
ds_info = tfds.builder("cats_vs_dogs").info
print(ds_info.version)

4.0.1


In [7]:
# Carrega os datasets do tensorflow.

ds_dogscats_train = tfds.load("cats_vs_dogs:4.0.1", split="train[:80%]", shuffle_files=True,
                              as_supervised=True)
ds_dogscats_test = tfds.load("cats_vs_dogs", split="train[80%:]", shuffle_files=True,
                             as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.T9Z4QS_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [8]:
# As dimenções do modelo não estão no formato suportado pelo modelo (None, 224, 224, 3).
# O código a seguir corrige isso, além de também normalizar as imagens.
def redimensiona_img(img, label):
  image = tf.image.resize(img, (224, 224))
  image = tf.cast(image, tf.float32) / 255.0
  label = tf.cast(label, tf.float32)
  return image, label

ds_dogscats_train = ds_dogscats_train.map(redimensiona_img).batch(32)
ds_dogscats_test = ds_dogscats_test.map(redimensiona_img).batch(32)

# Visualização de dados.

In [ ]:
# Ver as camadas do modelo final.
for i, camada in enumerate(modelo.layers):
  print(f"{i}: {camada.name}")

In [ ]:
# Ver dataset.
for i, dados in enumerate(ds_dogscats_train.take(1)):
  print(i, ':', dados)

In [ ]:
# Ver os rótulos e os recursos do dataset.
for recurso, rotulo in ds_dogscats_train.take(1):
  print(f"Recurso: {recurso}")
  print(f"Rótulo: {rotulo}")

In [ ]:
# Obter informações sobre os rótulos (do tipo ClassLabel)
print(ds_info.features["label"].names)

['cat', 'dog']


# Compilando o modelo.

In [9]:
# Compilando.
modelo.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Treinando o modelo.

In [ ]:
# Treinando.
history = modelo.fit(ds_dogscats_train, epochs=3)

# Salvar e carregar o modelo, porque o treinamento é longo demais.

Se não há um modelo salvo (que inclui arquitetura de camadas e pesos), então execute as células de **Fazer download do modelo**. Se não houver feito um treinamento, então execute as células das sessões para pré-processamento, configuração, compilação e treinamento do modelo, conforme instruídos no início, e então salve o arquivo.

Se um download já foi feito, então apenas execute as células de código em **Fazer upload do modelo**, a seguir.

As bibliotecas no topo do nootbook precisam ser carregadas antes de qualquer coisa.

## Fazer download dos pesos do modelo.

In [ ]:
# Salva os pesos.
modelo.save("modelo.keras")

In [ ]:
# Baixa localmente.
files.download("modelo.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Fazer upload dos pesos do modelo.

In [ ]:
# Fazer upload do modelo.
arquivo_modelo_salvo = files.upload()

Saving modelo.keras to modelo.keras


In [ ]:
# Carregar o modelo salvo.
modelo_carregado = tf.keras.models.load_model("modelo.keras")

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# O nootbook acessa o modelo treinado pela variável "modelo", mas é inconveniente procurar entre as demais
# células àquelas que criam uma variável assim. Então, crie uma variável "modelo" com o modelo carregado aqui.
modelo = modelo_carregado

# Função para converter as probabilidades retornadadas pelo modelo em classes (binárias)

In [ ]:
def converter_probabilidades_em_classes(probabilidades:tf.Tensor, limiar:float):
  classes = np.where(probabilidades > limiar, 1, 0)
  return classes

# Testar o modelo.

In [ ]:
# Ver a acurácia e perda quando usado nos dados de treinamento.
test_perda, test_acu = modelo.evaluate(ds_dogscats_test)

print(f"Acurácia: {test_acu}, perda: {test_perda}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.9892 - loss: 0.0380
Acurácia: 0.9871023297309875, perda: 0.04386857524514198


In [ ]:
# Realizar previsões e ver resultado.
prev = modelo.predict(ds_dogscats_test)

print(prev)

146/146 ━━━━━━━━━━━━━━━━━━━━ 189s 1s/step
[[2.4209965e-07]
 [9.9988502e-01]
 [5.1453281e-07]
 ...
 [8.8566852e-09]
 [9.9999899e-01]
 [3.6868742e-06]]


In [ ]:
# Convertes as probabilidades previstas em classificações.
# Usar o limiar padrão de 0.5 para alinhar com o cálculo de acurácia do modelo.evaluate
prev_bin = converter_probabilidades_em_classes(prev, 0.5)

print(prev_bin)

[[0]
 [1]
 [0]
 ...
 [0]
 [1]
 [0]]


In [ ]:
# Obter rótulos do conjunto de dados para teste.
# A concatenação é necessárias pelos exemplos do dataset estarem reunidos batchs (isto é, uma dimenção extra.)
y_true = np.concatenate([y for x, y in ds_dogscats_test.as_numpy_iterator()])
print("Formato: ", y_true.shape)
print(y_true)

Formato:  (4652,)
[0. 1. 0. ... 0. 1. 0.]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Matriz de confusão.
cm = confusion_matrix(y_true, prev_bin)
TN, FP, FN, TP = cm.ravel()

print(f"Matriz de Confusão:\n{cm}")
print(f"TN: {TN}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"TP: {TP}")

# Métricas de Desempenho
acuracia = accuracy_score(y_true, prev_bin)
precisao = precision_score(y_true, prev_bin)
recall = recall_score(y_true, prev_bin)
f1 = f1_score(y_true, prev_bin)

print(f"\nAcurácia (Accuracy): {acuracia:.4f}")
print(f"Precisão (Precision): {precisao:.4f}")
print(f"Recall (Sensibilidade / TPR): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Matriz de Confusão:
[[2091  189]
 [ 195 2177]]
TN: 2091
FP: 189
FN: 195
TP: 2177

Acurácia (Accuracy): 0.9175
Precisão (Precision): 0.9201
Recall (Sensibilidade / TPR): 0.9178
F1-Score: 0.9190
